# **Phân tích, trực quan hoá, đưa ra insight**
- Ở bước này chúng ta sẽ cùng nhau tìm hiểu *câu chuyện* phía sau dữ liệu của chúng ta có những gì, đồng thời cũng rút ra những insight quan trọng từ dữ liệu.
- Chúng ta sẽ thực hiện phân tích dữ liệu qua các bước sau:
    - Thống kê mô tả dữ liệu: tập trung vào các đặc trưng (fatalities, injuries, landslide_category, landslide_trigger, ...) để hiểu rõ hơn về phân phối và các đặc điểm chính của chúng.
    - Phân tích địa lý: sử dụng các biểu đồ bản đồ để trực quan hóa phân bố địa lý của các sự kiện lở đất, từ đó nhận diện các khu vực có nguy cơ cao.
    - Phân tích theo thời gian: khám phá xu hướng theo thời gian của các sự kiện lở đất, bao gồm sự thay đổi theo năm, mùa và các yếu tố thời tiết
    - Phân tích mối quan hệ giữa các biến: sử dụng các biểu đồ tương quan để tìm hiểu mối quan hệ giữa các biến khác nhau trong dữ liệu.
    - Cuối cùng, chúng ta sẽ tổng hợp các phát hiện quan trọng và đưa ra các kết luận từ phân tích dữ liệu.
- Trong các bước trên, chúng ta cũng tích hợp trả lời các câu hỏi có ý nghĩa.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px


import sys
import os
sys.path.append(os.path.abspath(r"..\src"))

import config as cf
import data_processing as dp 
import utils as ut 

In [ ]:
df = pd.read_csv(cf.RAW_DATA)
print(f"Dữ liệu có kích thước: {df.shape}")
df.head(3)

## **Những thứ râu ria**

In [ ]:
cols_to_drop = [
    #'event_id',
    'event_import_id', 
    'event_import_source', 
    'created_date', 
    'submitted_date', 
    'last_edited_date',
    'source_link', 
    'photo_link', 
    'storm_name',
    'country_code', 
    'event_time', 
    'notes',
    'gazeteer_closest_point', 
    'gazeteer_distance'
]

# Xóa cột 
df.drop(columns=[c for c in cols_to_drop if c in df.columns], inplace=True)

print(f"Kích thước sau khi xóa cột thừa: {df.shape}")


In [ ]:
# Chuyển đổi sang datetime, các giá trị lỗi sẽ biến thành NaT
df['event_date'] = pd.to_datetime(df['event_date'], format='%m/%d/%Y %I:%M:%S %p', errors='coerce')

print("Đã hoàn thành xử lý thời gian.")

In [ ]:
cols_to_fix = ['event_description', 'location_description', 
               'admin_division_name', 'event_title','source_name', 'admin_division_name', 'country_name']
for col in cols_to_fix:
    df[col] = df[col].apply(ut.fix_encoding)

print("Đã sửa xong lỗi font cho các cột văn bản.")

In [ ]:
df = df.apply(dp.fill_missing_locations, axis=1)
print("Hoàn tất")

In [ ]:
def clean_text(text):
    # Nếu là giá trị rỗng (NaN) thì trả về 'unknown'
    if pd.isna(text):
        return "unknown"
    
    # Chuyển về chữ thường và xoá khoảng trắng thừa ở 2 đầu
    return str(text).lower().strip()

# --- ÁP DỤNG ---
# Danh sách các cột bạn muốn xử lý
text_columns = ['landslide_category', 'landslide_trigger', 'landslide_size', 'country_name', 'admin_division_name']

for col in text_columns:
    if col in df.columns:
        df[col] = df[col].apply(clean_text)

# Kiểm tra nhanh kết quả
df[text_columns].head(100)

## **Question Formation**
### **1. Sạt lở đất có phải là một hiểm họa có tính quy luật không? Quy luật phân bố theo không gian và thời gian của nó như thế nào?**

In [ ]:
df['event_date'] = pd.to_datetime(
    df['event_date'],
    errors='coerce'
)
df['year'] = df['event_date'].dt.year
df['month'] = df['event_date'].dt.month

#### 1.1 Sạt lở có xảy ra đều theo thời gian hay tập trung vào 1 thời gian nhất định?


In [ ]:

# --- A. THỐNG KÊ CƠ BẢN ---
print("=== THỐNG KÊ THỜI GIAN ===")
print(f"Dữ liệu từ ngày: {df['event_date'].min().date()} đến {df['event_date'].max().date()}")
print(f"Tổng khoảng thời gian: {(df['event_date'].max() - df['event_date'].min()).days} ngày")
print(f"Năm có nhiều sạt lở nhất: {df['year'].mode()[0]} ({df['year'].value_counts().max()} vụ)")
print(f"Tháng 'đen tối' nhất trong năm: Tháng {df['month'].mode()[0]}")

# --- B. TRỰC QUAN HÓA - Xu hướng theo năm ---
plt.figure(figsize=(14, 8))

# Biểu đồ 1: Xu hướng theo Năm (Yearly Trend)
sns.countplot(data=df, x='year', color='steelblue')
plt.title('Xu hướng số vụ sạt lở qua các năm (Yearly Trend)', fontsize=16, fontweight='bold', pad=20)
plt.ylabel('Số lượng vụ', fontsize=12)
plt.xlabel('Năm', fontsize=12)
plt.xticks(rotation=45)  # Xoay nhãn năm cho dễ đọc
plt.grid(axis='y', alpha=0.3, linestyle='--')

# Thêm giá trị trên mỗi cột (tùy chọn)
for container in plt.gca().containers:
    plt.gca().bar_label(container, fmt='%.0f', fontsize=9, padding=3)

plt.tight_layout()
plt.show()

- Ta thấy bộ dữ liệu được thu thập từ năm 1988 - 2017 nhưng có sự tập trung từ năm 2007 nên nhóm em sẽ lấy dữ liệu từ năm 2007 - 2017
- Xét về tổng thể, sạt lở có xu hướng tăng qua các năm, có những năm có số vụ sạt lở cao đột biến 

In [ ]:
df_final = df[
    (df['event_date'].dt.year >= 2007) & 
    (df['event_date'].dt.year <= 2017)
].copy()

df = df_final

#### 1.3 Có sự khác biệt giữa các vùng khí hậu hay không? Có theo tính mùa vụ hay không?


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import calendar

# 1. Chuyển đổi cột ngày tháng sang datetime (nếu chưa làm)
# errors='coerce' sẽ biến các giá trị lỗi thành NaT (Not a Time)
df['event_date'] = pd.to_datetime(df['event_date'], errors='coerce')

# Loại bỏ các dòng không có ngày tháng
df_time = df.dropna(subset=['event_date']).copy()

# 2. Trích xuất Tháng và Năm
df_time['month'] = df_time['event_date'].dt.month
df_time['year'] = df_time['event_date'].dt.year

# 3. Thống kê số vụ theo tháng (Tổng hợp 10 năm)
monthly_counts = df_time['month'].value_counts().sort_index()

# Tạo tên tháng (Jan, Feb...) thay vì số 1, 2...
month_names = [calendar.month_abbr[i] for i in monthly_counts.index]

# --- VẼ BIỂU ĐỒ ---
fig, axes = plt.subplots(1, 2, figsize=(20, 7))

# Biểu đồ 1: Phân bố theo Tháng (Seasonality)
sns.barplot(x=monthly_counts.index, y=monthly_counts.values, ax=axes[0], palette='viridis')
axes[0].set_xticklabels(month_names)
axes[0].set_title('Tổng số vụ Sạt lở theo Tháng (2007-2017)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Tháng')
axes[0].set_ylabel('Tổng số vụ')
axes[0].grid(axis='y', linestyle='--', alpha=0.5)

# Thêm nhãn giá trị
for container in axes[0].containers:
    axes[0].bar_label(container, padding=3)

# Biểu đồ 2: Heatmap Năm vs Tháng (Chi tiết hoá)
# Tạo bảng Pivot: Hàng = Năm, Cột = Tháng, Giá trị = Số vụ
pivot_table = df_time.pivot_table(index='year', columns='month', values='event_id', aggfunc='count', fill_value=0)
pivot_table.columns = [calendar.month_abbr[i] for i in pivot_table.columns] # Đổi tên cột thành tên tháng

sns.heatmap(pivot_table, annot=True, fmt='d', cmap='YlOrRd', ax=axes[1], cbar_kws={'label': 'Số vụ sạt lở'})
axes[1].set_title('Bản đồ nhiệt: Tần suất Sạt lở theo Thời gian', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Năm')
axes[1].set_xlabel('Tháng')

plt.tight_layout()
plt.show()

- KHÔNG ĐỀU. Sạt lở đất có tính tập trung rất cao theo mùa.
- ta nhận ra có sự thay đổi, thời gian trước 2010 các vụ sạt lở trên thế giới xảy ra không nhiều ( < 90 vụ). Nhưng bắt đầu từ năm 2010, các vụ sạt lở xảy ra trên thế giới có sự thay đổi lớn.
    - từ tháng 3/2010 - 8/2011: Trong suốt thời gian này các vụ sạt lở trên thế giới xảy ra liên tục, mỗi tháng đều có số vụ trung bình >100 vụ. Sau giai đoạn này trở về như trước năm 2010
    - 6/2013 - 9/2013: Đây là giai đoạn mà số vụ cao bất thường
    - 1/2017 - 7/2017: số vụ cao bất thường, có sự biến động lớn
- Về tổng thế số vụ sạt lở sẽ tăng cao ở giai đoạn tháng 6,7,8  và giai đoạn cuối năm (12, 1). Ta có thể thấy được rằng số vụ sạt lở trên toàn thế giới có thể xảy ra theo mùa vụ ở những khu vực khác nhau tuy thuộc vào vùng khí hậu (cần phân tích thêm), Ta còn được thấy vào tháng 3, thì số vụ ở đây tăng rõ rệt, cần tìm hiểu thêm.

In [ ]:
import pandas as pd
import numpy as np

# --- BƯỚC 1: TẠO CỘT KHÍ HẬU (5 ZONES) ---
def get_5_climate_zones(lat):
    # Trả về Unknown nếu không có toạ độ
    if pd.isna(lat): 
        return 'unknown'
    # Lấy trị tuyệt đối để tính chung cho cả Bắc và Nam bán cầu
    abs_lat = abs(lat)
    
    # Phân loại theo 5 vùng
    if abs_lat <= 23.5:
        return 'tropical'      # 0 - 23.5
    elif abs_lat <= 35:
        return 'subtropical' # 23.5 - 35
    elif abs_lat <= 60:
        return 'temperate'        # 35 - 60
    elif abs_lat <= 66.5:
        return 'subpolar'    # 60 - 66.5
    else:
        return 'polar'           # > 66.5

# Áp dụng hàm vào cột latitude
df['climate_zone'] = df['latitude'].apply(get_5_climate_zones)

# Lọc bỏ dữ liệu Unknown
df_climate = df[df['climate_zone'] != 'unknown'].copy()

# Kiểm tra kết quả phân bố
print("--- SỐ LƯỢNG VỤ SẠT LỞ THEO 5 VÙNG KHÍ HẬU ---")
print(df_climate['climate_zone'].value_counts())

In [ ]:
# --- 2. VẼ BIỂU ĐỒ (2 Subplots) ---
fig, axes = plt.subplots(1, 2, figsize=(20, 7))

# Thứ tự hiển thị trên biểu đồ (từ Xích đạo về Cực)
order_zones = ['tropical', 'subtropical', 'temperate', 'subpolar', 'polar']
existing_order = [z for z in order_zones if z in df_climate['climate_zone'].unique()]

# --- Biểu đồ 1: Số lượng vụ sạt lở (Bar Chart) ---
sns.countplot(
    data=df_climate, 
    x='climate_zone', 
    order=existing_order, 
    ax=axes[0], 
    palette='Set1'
)
axes[0].set_title('Tổng số vụ Sạt lở theo Vùng Khí hậu (2007-2017)', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Tổng số vụ')
axes[0].set_xlabel('Vùng Khí hậu')
axes[0].grid(axis='y', linestyle='--', alpha=0.5)
# Thêm nhãn số lượng trên cột
for container in axes[0].containers:
    axes[0].bar_label(container, padding=3, fontsize=10)

# --- Biểu đồ 2: Xu hướng Mùa vụ (Line Chart) ---
# Thống kê số vụ theo Tháng và Vùng
zone_monthly = df_climate.groupby(['climate_zone', df_climate['event_date'].dt.month])['event_id'].count().reset_index()
zone_monthly.columns = ['Climate Zone', 'Month', 'Event Count']

sns.lineplot(
    data=zone_monthly, 
    x='Month', 
    y='Event Count', 
    hue='Climate Zone', 
    hue_order=existing_order,
    style='Climate Zone',
    markers=True, 
    dashes=False,
    linewidth=2.5,
    palette='Set1',
    ax=axes[1]
)
axes[1].set_title('Tính Mùa vụ: Phân bố số vụ theo Tháng', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Tháng trong năm')
axes[1].set_ylabel('Số vụ trung bình/tháng')
axes[1].set_xticks(range(1, 13))
axes[1].grid(True, linestyle='--', alpha=0.5)
axes[1].legend(title='Vùng Khí hậu')

plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# 1. CHUẨN BỊ DỮ LIỆU
# Đảm bảo cột thời gian là datetime
df['event_date'] = pd.to_datetime(df['event_date'], errors='coerce')
df['year'] = df['event_date'].dt.year

# Danh sách 3 vùng mục tiêu cần vẽ
target_zones = ['tropical', 'subtropical', 'temperate']

# Lọc dữ liệu: Chỉ lấy 3 vùng này VÀ loại bỏ các năm thiếu dữ liệu (nếu cần)
# Ở đây mình lấy từ năm 2007 trở đi cho dữ liệu ổn định
df_trend = df[
    (df['climate_zone'].isin(target_zones)) & 
    (df['year'] >= 2007)
].copy()

# Đếm số vụ theo Năm và Vùng
yearly_counts = df_trend.groupby(['year', 'climate_zone']).size().reset_index(name='count')

# 2. VẼ BIỂU ĐỒ ĐƯỜNG (LINE CHART)
plt.figure(figsize=(12, 7))

# Định nghĩa màu sắc đặc trưng cho khí hậu
# Nhiệt đới = Đỏ (Nóng), Cận nhiệt = Cam, Ôn đới = Xanh (Mát)
colors = {'tropical': '#FF4500', 'subtropical': '#FF8C00', 'temperate': '#4682B4'}

sns.lineplot(
    data=yearly_counts,
    x='year',
    y='count',
    hue='climate_zone',    # Phân nhóm theo vùng
    palette=colors,        # Gán màu
    style='climate_zone',  # Mỗi vùng 1 kiểu nét (nét liền, đứt...) để dễ phân biệt
    markers=True,          # Hiện chấm tròn tại các năm
    dashes=False,          # (Tuỳ chọn) Vẽ nét liền hết cho đẹp
    linewidth=3,           # Đường đậm
    markersize=8
)

# 3. TRANG TRÍ
plt.title('Biến động Số vụ Sạt lở theo Năm giữa 3 Vùng Khí hậu', fontsize=16, fontweight='bold')
plt.xlabel('Năm', fontsize=12)
plt.ylabel('Số lượng vụ sạt lở', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.5)

# Hiển thị chú thích (Legend) rõ ràng
plt.legend(title='Vùng khí hậu', title_fontsize=12, fontsize=11)

# Đảm bảo trục X hiện đủ các năm
plt.xticks(yearly_counts['year'].unique())

plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# --- 1. CHUẨN BỊ DỮ LIỆU ---
# Đảm bảo cột thời gian và khí hậu
df['event_date'] = pd.to_datetime(df['event_date'], errors='coerce')
df['month'] = df['event_date'].dt.month

# Lọc 3 vùng chính và chuẩn hoá tên
target_zones = ['tropical', 'subtropical', 'temperate']
df['climate_zone_clean'] = df['climate_zone'].astype(str).str.lower().str.strip()
df_rose = df[df['climate_zone_clean'].isin(target_zones)].copy()

# Gom nhóm đếm số vụ theo (Vùng, Tháng)
monthly_counts = df_rose.groupby(['climate_zone_clean', 'month']).size().reset_index(name='event_count')

# --- 2. THIẾT LẬP THÔNG SỐ VẼ BIỂU ĐỒ CỰC ---
# Số lượng tháng
N = 12
# Tạo các góc (theta) cho 12 tháng (tính bằng radians)
# linspace tạo ra N điểm từ 0 đến 2*pi (vòng tròn)
theta = np.linspace(0.0, 2 * np.pi, N, endpoint=False)
# Độ rộng của mỗi cánh hoa (bar)
width = (2 * np.pi) / N

# Cấu hình màu sắc và tiêu đề Việt hóa
colors = {'tropical': '#FF6B6B', 'subtropical': '#FFD93D', 'temperate': '#4D96FF'}
titles = {'tropical': 'Nhiệt đới (Tropical)', 'subtropical': 'Cận nhiệt đới (Subtropical)', 'temperate': 'Ôn đới (Temperate)'}
zone_order = ['tropical', 'subtropical', 'temperate']
month_labels = ['Th1', 'Th2', 'Th3', 'Th4', 'Th5', 'Th6', 'Th7', 'Th8', 'Th9', 'Th10', 'Th11', 'Th12']

# --- 3. VẼ 3 BIỂU ĐỒ (SUBPLOTS VỚI POLAR PROJECTION) ---
# subplot_kw=dict(polar=True) là chìa khoá để biến trục vuông thành trục tròn
fig, axes = plt.subplots(1, 3, figsize=(20, 7), subplot_kw=dict(polar=True))

# Tìm giá trị max tổng thể để đồng bộ trục bán kính (giúp so sánh độ lớn giữa các vùng)
max_radius = monthly_counts['event_count'].max()

for i, zone in enumerate(zone_order):
    ax = axes[i]
    
    # Lấy dữ liệu của vùng
    zone_data = monthly_counts[monthly_counts['climate_zone_clean'] == zone].set_index('month')
    
    # QUAN TRỌNG: Reindex để đảm bảo đủ 12 tháng, tháng thiếu điền 0
    zone_data = zone_data.reindex(range(1, 13), fill_value=0)
    radii = zone_data['event_count'].values # Chiều dài bán kính (số vụ)

    # Vẽ biểu đồ cột trên hệ toạ độ cực
    bars = ax.bar(theta, radii, width=width, bottom=0.0, color=colors[zone], alpha=0.7, edgecolor='black')

    # --- TRANG TRÍ TRỤC CỰC ---
    # Đặt hướng 0 độ (Tháng 1) lên đỉnh (hướng Bắc)
    ax.set_theta_zero_location("N")
    # Đặt chiều quay theo chiều kim đồng hồ
    ax.set_theta_direction(-1)
    
    # Đặt nhãn cho các tháng xung quanh vòng tròn
    ax.set_xticks(theta)
    ax.set_xticklabels(month_labels, fontsize=10, fontweight='bold')
    
    # Đặt giới hạn cho trục bán kính (để dễ so sánh giữa 3 hình)
    # Cộng thêm chút đỉnh cho thoáng
    ax.set_ylim(0, max_radius * 1.1)
    
    # Chỉnh lưới và nhãn trục bán kính
    ax.grid(True, linestyle=':', color='gray')
    # Chỉ hiện vài mốc bán kính để đỡ rối
    ax.set_yticks(np.linspace(0, max_radius, 4)[1:]) # Bỏ số 0 ở tâm
    ax.tick_params(axis='y', labelsize=9, colors='gray')

    ax.set_title(titles[zone], fontsize=14, fontweight='bold', pad=20)

plt.suptitle('Biểu đồ Hoa hồng (Rose Chart): Phân bố Mùa vụ Sạt lở theo Vùng Khí hậu', fontsize=16, y=1.05)
plt.tight_layout()
plt.show()

- nhận xét:
  Vùng Nhiệt đới (Tropical - Đỏ):

Có hai đỉnh sạt lở rõ rệt trong năm:

Đỉnh thứ nhất: Vào khoảng tháng 8 (nằm trong cụm tháng 7-8-9).

Đỉnh thứ hai và cao hơn: Vào tháng 10 (nằm trong cụm tháng 9-10-11).

Nhận xét: Điều này phản ánh rất rõ hai mùa mưa điển hình của nhiều khu vực nhiệt đới (ví dụ: mùa mưa chính và mùa mưa phụ, hoặc ảnh hưởng của hai đợt gió mùa khác nhau). Tháng 10 thường là thời điểm mưa lũ cuối mùa, có thể gây sạt lở nghiêm trọng.

Vùng Cận nhiệt đới (Subtropical - Xanh dương):

Có một đỉnh duy nhất và rõ nét vào tháng 7 (trong cụm tháng 6-7-8).

Nhận xét: Đây là đặc trưng của khí hậu cận nhiệt đới với một mùa mưa tập trung vào giữa mùa hè, thường liên quan đến gió mùa Đông Á hoặc các hiện tượng mưa lớn do áp thấp nhiệt đới, bão.

Vùng Ôn đới (Temperate - Xanh lá):

Tần suất sạt lở cao rơi vào các tháng mùa đông và đầu xuân: cụ thể là tháng 12, tháng 1, và một đợt tăng cao khác vào tháng 3.

Nhận xét: Điều này hoàn toàn khác với suy nghĩ thông thường (cho rằng sạt lở nhiều vào mùa hè). Nguyên nhân chính có thể là:

Mưa phùn kéo dài hoặc mưa rào mùa đông làm bão hòa đất.

Quá trình đóng băng và tan băng (frost-thaw cycle) vào cuối đông, đầu xuân (tháng 3) làm mất ổn định sườn dốc.

Bão mùa đông ở một số khu vực ôn đới.

Vùng Cận cực & Cực đới (Subpolar & Polar - Tím và màu nhạt):

Số vụ không đáng kể quanh năm, có thể có một chút dao động nhỏ nhưng không tạo thành đỉnh rõ rệt.

#### 1.4 Các vùng trên thế giới thì sao?


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# 1. ĐỊNH NGHĨA HÀM PHÂN VÙNG (Giữ nguyên logic của bạn)
def get_geo_region(row):
    lat = row['latitude']
    lon = row['longitude']
    
    # Nếu thiếu toạ độ thì trả về Unknown để tránh lỗi
    if pd.isna(lat) or pd.isna(lon):
        return None

    # --- KHU VỰC CHÂU MỸ (Kinh độ < -30) ---
    # --- KHU VỰC CHÂU MỸ (Kinh độ < -30) ---
    if lon < -30:
        if lat > 8:
            return 'north_america'  # Bao gồm Canada, Mỹ, Mexico, Trung Mỹ và Caribbean
        else:
            return 'south_america'   # Từ Colombia trở xuống
            
    # --- KHU VỰC CHÂU ÂU & PHI (Kinh độ từ -30 đến 60) ---
    elif lon < 60:
        if lat > 30: return 'europe'
        else: return 'africa'
            
    # --- KHU VỰC CHÂU Á & ÚC (Kinh độ >= 60) ---
    else:
        if lon < 95: return 'south_asia'
        else:
            if lat > 25: return 'east_asia'
            else: return 'se_asia_oceania'

# 2. XỬ LÝ DỮ LIỆU TRÊN DF GỐC
# Đảm bảo cột ngày tháng chuẩn
# df['event_date'] = pd.to_datetime(df['event_date'], errors='coerce')
# df['month'] = df['event_date'].dt.month

# Áp dụng hàm phân vùng
print("Đang phân vùng dữ liệu...")
df['region'] = df.apply(get_geo_region, axis=1)

# LỌC DỮ LIỆU ĐỂ VẼ (Bỏ những dòng không phân vùng được hoặc thiếu ngày)
df_plot = df.dropna(subset=['region', 'month', 'latitude', 'longitude']).copy()

# 3. VẼ BIỂU ĐỒ
fig, axes = plt.subplots(2, 1, figsize=(14, 13))

# --- HÌNH 1: BẢN ĐỒ KIỂM TRA ---
sns.scatterplot(
    data=df_plot, 
    x='longitude', 
    y='latitude', 
    hue='region', 
    palette='tab10', 
    s=15, 
    ax=axes[0]
)
axes[0].set_title('Phân vùng Địa lý theo Luật Kinh/Vĩ độ (Manual Segmentation)', fontsize=15, fontweight='bold')
axes[0].grid(True, linestyle='--', alpha=0.5)

# Vẽ các đường ranh giới cắt tượng trưng (Để báo cáo nhìn cho xịn)
axes[0].axvline(-30, color='red', linestyle='--', alpha=0.5) # Ranh giới Đại Tây Dương
axes[0].axvline(60, color='red', linestyle='--', alpha=0.5)  # Ranh giới Ấn Độ Dương


# --- HÌNH 2: HEATMAP MÙA VỤ ---
# Tạo bảng pivot từ df_plot
pivot_manual = df_plot.pivot_table(
    index='region', 
    columns='month', 
    values='event_id', 
    aggfunc='count'
).fillna(0)

# Sắp xếp thứ tự vùng địa lý (Từ Tây sang Đông cho logic)
ordered_regions = [
    'north_america', 'south_america',
    'europe', 'africa',
    'south_asia', 'east_asia', 'se_asia_oceania'
]
# Chỉ lấy các vùng thực tế có trong dữ liệu
valid_order = [r for r in ordered_regions if r in pivot_manual.index]
pivot_manual = pivot_manual.loc[valid_order]

sns.heatmap(
    pivot_manual, 
    annot=True, 
    fmt='.0f', 
    cmap='YlOrRd', 
    linewidths=.5,
    cbar_kws={'label': 'Số lượng vụ sạt lở'},
    ax=axes[1]
)
axes[1].set_title('Tính Mùa vụ Sạt lở theo Khu vực Địa lý (Chi tiết)', fontsize=15, fontweight='bold')
axes[1].set_xlabel('Tháng')
axes[1].set_ylabel('')

plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# 1. TÍNH TOÁN SỐ LƯỢNG
# Đếm số vụ theo vùng và sắp xếp giảm dần
region_counts = df['region'].value_counts().reset_index()
region_counts.columns = ['Region', 'Total_Events']

# Lọc bỏ 'Unknown' hoặc None nếu có để biểu đồ sạch
region_counts = region_counts[region_counts['Region'] != 'unknown']

# 2. VẼ BIỂU ĐỒ CỘT NGANG
plt.figure(figsize=(12, 8))

# Dùng màu gradient (đậm nhạt) để nhấn mạnh vùng nhiều nhất
ax = sns.barplot(
    data=region_counts,
    y='Region',
    x='Total_Events',
    palette='viridis' # Màu từ Vàng sang Tím xanh
)

# 3. HIỂN THỊ SỐ LIỆU TRÊN CỘT
for container in ax.containers:
    ax.bar_label(container, padding=5, fontsize=11, fontweight='bold')

# Trang trí
plt.title('Tổng số vụ Sạt lở ghi nhận theo Khu vực Địa lý', fontsize=16, fontweight='bold')
plt.xlabel('Tổng số vụ (Events)', fontsize=12)
plt.ylabel('Khu vực (Region)', fontsize=12)
plt.grid(axis='x', linestyle='--', alpha=0.5)

# Tăng khoảng cách lề phải để số không bị cắt
plt.xlim(0, region_counts['Total_Events'].max() * 1.15)

plt.tight_layout()
plt.show()

# 4. IN BẢNG SỐ LIỆU
print("--- BẢNG THỐNG KÊ CHI TIẾT ---")
# Tính thêm % đóng góp
region_counts['Percentage (%)'] = (region_counts['Total_Events'] / region_counts['Total_Events'].sum()) * 100
display(region_counts.round(1))

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import math

# --- 1. CHUẨN BỊ DỮ LIỆU ---
# Đảm bảo bạn đã chạy bước tạo cột 'region' ở trên
df_rose = df.dropna(subset=['region', 'month']).copy()

# Sắp xếp thứ tự vùng cho logic (Từ Tây sang Đông)
# ordered_regions = [
#     'north_america', 'Central America (Trung Mỹ)', 'South America (Nam Mỹ)',
#     'Europe (Châu Âu)', 'Africa (Châu Phi)',
#     'South Asia (Nam Á)', 'East Asia (Đông Á)', 'SE Asia & Oceania (ĐNÁ & Úc)'
# ]
# Chỉ lấy các vùng có dữ liệu thực tế
valid_regions = [r for r in ordered_regions if r in df_rose['region'].unique()]

# --- 2. THIẾT LẬP THÔNG SỐ CHUNG ---
N = 12
theta = np.linspace(0.0, 2 * np.pi, N, endpoint=False)
width = (2 * np.pi) / N
month_labels = ['Th1', 'Th2', 'Th3', 'Th4', 'Th5', 'Th6', 'Th7', 'Th8', 'Th9', 'Th10', 'Th11', 'Th12']

# Tính số dòng/cột (Vẽ 2 hàng, 4 cột)
n_cols = 4
n_rows = math.ceil(len(valid_regions) / n_cols)

# --- 3. VẼ BIỂU ĐỒ ---
fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, 10), subplot_kw=dict(polar=True))
axes = axes.flatten()
palette = sns.color_palette("tab10", len(valid_regions))

for i, region in enumerate(valid_regions):
    ax = axes[i]
    color = palette[i]
    
    # Lấy dữ liệu của vùng
    region_data = df_rose[df_rose['region'] == region].groupby('month').size()
    # Reindex đủ 12 tháng
    region_data = region_data.reindex(range(1, 13), fill_value=0)
    radii = region_data.values

    # Vẽ cột
    bars = ax.bar(theta, radii, width=width, bottom=0.0, color=color, alpha=0.6, edgecolor='black', linewidth=0.8)

    # --- TÙY CHỈNH TRỤC SỐ (GRID LINES) ---
    # Tính giá trị max của vùng hiện tại để chia lưới cho đẹp
    local_max = region_data.max()
    
    # Tạo 4 mốc lưới: ví dụ max là 100 thì lưới là [25, 50, 75, 100]
    # Làm tròn số grid cho đẹp
    if local_max < 5: 
        grids = np.arange(1, local_max + 1, 1) # Nếu ít quá thì chia từng đơn vị
    else:
        # Chia làm 3-4 vòng tròn đồng tâm
        grids = np.linspace(0, local_max, 5)[1:] # Bỏ số 0
        grids = np.round(grids).astype(int) # Làm tròn thành số nguyên

    # Thiết lập các vòng lưới
    ax.set_yticks(grids)
    
    # Format số hiển thị trên lưới (Màu xám, font nhỏ)
    # set_rlabel_position(45): Xoay trục số nghiêng 45 độ để đỡ che mất chữ Tháng 1 (Bắc)
    ax.set_rlabel_position(45) 
    ax.tick_params(axis='y', labelsize=8, colors='black', pad=0)
    
    # --- TRANG TRÍ KHÁC ---
    ax.set_theta_zero_location("N") # Tháng 1 hướng Bắc
    ax.set_theta_direction(-1)      # Chiều kim đồng hồ
    
    ax.set_xticks(theta)
    ax.set_xticklabels(month_labels, fontsize=9, fontweight='bold')
    ax.set_title(region, fontsize=12, fontweight='bold', pad=25, color='darkblue')
    
    # Giới hạn trục Y (cộng thêm chút đỉnh cho số không bị cắt)
    ax.set_ylim(0, local_max * 1.15)
    
    # Màu lưới
    ax.grid(True, linestyle='--', alpha=0.5, color='gray')

# Xoá biểu đồ thừa
for j in range(len(valid_regions), len(axes)):
    fig.delaxes(axes[j])

plt.suptitle('Biểu đồ Hoa hồng Mùa vụ theo Vùng (Kèm số lượng trên lưới)', fontsize=18, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math

# 1. CHUẨN BỊ DỮ LIỆU
# Đảm bảo có cột Year và Region
df['event_date'] = pd.to_datetime(df['event_date'], errors='coerce')
df['year'] = df['event_date'].dt.year

# Lọc dữ liệu sạch
df_trend = df.dropna(subset=['region', 'year']).copy()
# Chỉ lấy giai đoạn ổn định (ví dụ 2007-2017) để tránh các năm dữ liệu thiếu
df_trend = df_trend[(df_trend['year'] >= 2007) & (df_trend['year'] <= 2017)]

# Sắp xếp thứ tự vùng
# ordered_regions = [
#     'North America (Bắc Mỹ)', 'Central America (Trung Mỹ)', 'South America (Nam Mỹ)',
#     'Europe (Châu Âu)', 'Africa (Châu Phi)',
#     'South Asia (Nam Á)', 'East Asia (Đông Á)', 'SE Asia & Oceania (ĐNÁ & Úc)'
# ]
valid_regions = [r for r in ordered_regions if r in df_trend['region'].unique()]

# Tính số vụ theo năm cho từng vùng
yearly_region_counts = df_trend.groupby(['region', 'year']).size().reset_index(name='count')

# 2. THIẾT LẬP SUBPLOTS
n_cols = 4
n_rows = math.ceil(len(valid_regions) / n_cols)
fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, 10))
axes = axes.flatten()

# Dùng bộ màu chuẩn
palette = sns.color_palette("tab10", len(valid_regions))

# 3. VẼ BIỂU ĐỒ
for i, region in enumerate(valid_regions):
    ax = axes[i]
    color = palette[i]
    
    # Lấy dữ liệu vùng
    data = yearly_region_counts[yearly_region_counts['region'] == region]
    
    # Vẽ Line plot
    sns.lineplot(
        data=data, 
        x='year', 
        y='count', 
        ax=ax, 
        color=color, 
        marker='o',       # Chấm tròn
        linewidth=2.5,    # Đường đậm
        markersize=8
    )
    
    # --- TRANG TRÍ ---
    ax.set_title(region, fontsize=12, fontweight='bold', color='darkblue')
    ax.set_xlabel('')
    ax.set_ylabel('Số vụ')
    ax.grid(True, linestyle='--', alpha=0.5)
    
    # Hiển thị đủ các năm trên trục X (xoay 45 độ cho đỡ dính)
    ax.set_xticks(range(2007, 2018))
    ax.tick_params(axis='x', rotation=45, labelsize=9)
    
    # QUAN TRỌNG: Hiển thị giá trị (số vụ) ngay trên điểm marker
    for x, y in zip(data['year'], data['count']):
        ax.text(x, y + (data['count'].max()*0.05), # Nhích lên trên điểm 1 chút
                f'{int(y)}', 
                ha='center', va='bottom', fontsize=9, fontweight='bold', color=color)
        
    # Set giới hạn trục Y (để số không bị cắt)
    ax.set_ylim(0, data['count'].max() * 1.2)

# Xoá biểu đồ thừa
for j in range(len(valid_regions), len(axes)):
    fig.delaxes(axes[j])

plt.suptitle('Biến động Số vụ Sạt lở qua các năm theo Khu vực Địa lý (2007-2017)', fontsize=18, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 1. CHUẨN BỊ DỮ LIỆU
# Đảm bảo cột thời gian và region
df['event_date'] = pd.to_datetime(df['event_date'], errors='coerce')
df['year'] = df['event_date'].dt.year

# Lọc dữ liệu sạch & lấy giai đoạn 2007-2017
df_trend = df.dropna(subset=['region', 'year']).copy()
df_trend = df_trend[(df_trend['year'] >= 2007) & (df_trend['year'] <= 2017)]

# Tính số vụ theo năm cho từng vùng
yearly_region_counts = df_trend.groupby(['region', 'year']).size().reset_index(name='count')

# Sắp xếp để vùng nào nhiều vụ nhất thì vẽ sau (nổi lên trên) hoặc gán màu nổi
# Tính tổng vụ để sắp xếp Legend cho đẹp
total_counts = yearly_region_counts.groupby('region')['count'].sum().sort_values(ascending=False).index
yearly_region_counts['region'] = pd.Categorical(yearly_region_counts['region'], categories=total_counts, ordered=True)

# 2. VẼ BIỂU ĐỒ GỘP
plt.figure(figsize=(14, 8))

# Vẽ Line Plot với hue='region'
sns.lineplot(
    data=yearly_region_counts, 
    x='year', 
    y='count', 
    hue='region', 
    style='region', # Thêm style (nét đứt/liền) để dễ phân biệt hơn nếu in trắng đen
    markers=True,   # Hiện chấm tròn
    dashes=False,   # Muốn tất cả là nét liền cho dễ nhìn thì để False
    linewidth=2.5,
    palette='tab10',
    markersize=8
)

# 3. TRANG TRÍ
plt.title('So sánh Xu hướng Sạt lở giữa các Khu vực Địa lý (2007-2017)', fontsize=16, fontweight='bold')
plt.xlabel('Năm', fontsize=12)
plt.ylabel('Số lượng vụ sạt lở', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.5)

# Hiện đủ năm trên trục X
plt.xticks(range(2007, 2018))

# Đưa Legend ra ngoài biểu đồ (bên phải) để không che đường
plt.legend(title='Khu vực (Sắp xếp theo độ lớn)', bbox_to_anchor=(1.01, 1), loc='upper left', borderaxespad=0)

plt.tight_layout()
plt.show()

#### 1.5 Phân bố không gian của các vụ sạt lở có tập trung theo khu vực địa lý nhất định hay không?

In [ ]:
import plotly.express as px

# 1. CHUẨN BỊ DỮ LIỆU
# Đảm bảo cột climate_zone đã được tạo từ bước trước
# Loại bỏ giá trị 'unknown' hoặc NaN để bản đồ sạch đẹp hơn (tuỳ chọn)
df_map = df[df['climate_zone'] != 'unknown'].copy()

# 2. CẤU HÌNH MÀU SẮC (Color Palette)
# Định nghĩa màu theo nhiệt độ cho trực quan:
# Tropical (Nóng) -> Đỏ
# Subtropical (Ấm) -> Cam/Vàng
# Temperate (Ôn hoà) -> Xanh lá
# Subpolar (Lạnh) -> Xanh dương nhạt
# Polar (Băng giá) -> Xanh dương đậm/Tím
climate_colors = {
    'tropical': '#FF3333',    # Đỏ tươi
    'subtropical': '#FFA500', # Cam
    'temperate': '#32CD32',   # Xanh lá
    'subpolar': '#00BFFF',    # Xanh dương sáng
    'polar': '#00008B',       # Xanh dương đậm
    'unknown': 'gray'
}

# 3. VẼ BIỂU ĐỒ VỚI MAPBOX
fig = px.scatter_mapbox(
    df_map,
    lat="latitude",
    lon="longitude",
    
    # --- QUAN TRỌNG: TÔ MÀU THEO KHÍ HẬU ---
    color="climate_zone", 
    
    # Gán bộ màu đã định nghĩa ở trên
    color_discrete_map=climate_colors,
    
    # Sắp xếp thứ tự trong chú thích (Legend) từ Nóng -> Lạnh
    category_orders={"climate_zone": ["tropical", "subtropical", "temperate", "subpolar", "polar"]},

    # --- CẤU HÌNH HIỂN THỊ ---
    hover_name="event_title",
    hover_data={
        "latitude": False,
        "longitude": False,
        "country_name": True,
        "climate_zone": True,  # Hiển thị thêm vùng khí hậu khi rê chuột
        "fatality_count": True,
    },
    
    # Chọn style bản đồ
    mapbox_style="open-street-map", 
    zoom=1, 
    height=850, 
    title="Bản đồ Phân bố Sạt lở theo 5 Vùng Khí hậu (Tropical -> Polar)"
)

# 4. TINH CHỈNH ĐIỂM DỮ LIỆU
fig.update_traces(
    marker=dict(
        size=6,             # Kích thước chấm
        opacity=0.7         # Độ trong suốt để thấy mật độ chồng lấn
    )
)

# 5. TINH CHỈNH LAYOUT
fig.update_layout(
    margin={"r":0,"t":40,"l":0,"b":0},
    legend=dict(
        title="Vùng Khí hậu",
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01,
        bgcolor="rgba(255, 255, 255, 0.8)" # Nền trắng mờ cho khung chú thích dễ đọc
    )
)

fig.show()

### 2. "Sạt lở đất được ghi nhận tập trung ở đâu? Sự phân bố này phản ánh chính xác nguy cơ tự nhiên hay bị ảnh hưởng bởi khả năng quan sát và báo cáo của con người?"

#### 2.1 Quốc gia nào ghi nhận nhiều sạt lở nhất trong giai đoạn 2007 - 2017


In [ ]:

df_country = df[df['country_name'] != 'unknown'].copy()

# Top 10 nước nhiều vụ nhất
top_freq = df_country['country_name'].value_counts().head(10)

# Top 10 nước nhiều người chết nhất
top_fatal = df_country.groupby('country_name')['fatality_count'].sum().sort_values(ascending=False).head(10)

# Vẽ biểu đồ
plt.figure(figsize=(12, 7))

ax = sns.barplot(
    x=top_freq.values, 
    y=top_freq.index, 
    palette='Blues_r', 

)
ax.bar_label(ax.containers[0], padding=2, fontsize=11, fontweight='bold', color='darkblue')
# Thay vì chỉ dùng ax.containers[0], hãy lặp qua tất cả các container
# for container in ax.containers:
#     ax.bar_label(container, padding=2, fontsize=11, fontweight='bold', color='darkblue')
plt.title('Top 10 Countries with the Highest Number of Landslide', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Number of Landslide Events (count)', fontsize=12)
plt.ylabel('Country', fontsize=12)

plt.grid(axis='x', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()



#### 2.2 Theo cấp đơn vị hành chính thì sao?


In [ ]:
df_admin = df_country['admin_division_name']
top_admin_name = df_admin.value_counts().head(10)

plt.figure(figsize=(12, 7))

# 3. Vẽ biểu đồ thanh ngang

ax = sns.barplot(
    x=top_admin_name.values, 
    y=top_admin_name.index, 
    palette='Blues_r',
)
ax.bar_label(ax.containers[0], padding=5, fontsize=11, fontweight='bold', color='darkblue')

# 5. Trang trí tiêu đề và nhãn
plt.title('Top 10 Đơn vị hành chính có tần suất sạt lở cao nhất', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Số lượng vụ sạt lở (vụ)', fontsize=12)
plt.ylabel('Tên đơn vị hành chính', fontsize=12)

# Thêm lưới dọc để dễ so sánh độ dài
plt.grid(axis='x', linestyle='--', alpha=0.6)

# Tối ưu hóa không gian
plt.tight_layout()
plt.show()

#### 2.3 Liệu sự phân bố sạt lở quan sát được phản ánh hiện tượng tự nhiên hay thiên lệch do nguồn thu thập dữ liệu?


In [ ]:
# 1. Tính toán cơ bản
total_events = len(df)  # Tổng số vụ trong toàn bộ dữ liệu
source_counts = df['source_name'].value_counts() # Đếm số lượng từng nguồn

# 2. Lấy Top 10 và tính phần trăm
top_10_sources = source_counts.head(10)
top_10_pct = (top_10_sources / total_events) * 100

# 3. Tạo DataFrame đẹp để hiển thị bảng
stats_df = pd.DataFrame({
    'Source Name': top_10_sources.index,
    'Event Count': top_10_sources.values,
    'Contribution (%)': top_10_pct.values
})


# --- IN KẾT QUẢ THỐNG KÊ ---
print("--- BẢNG THỐNG KÊ CHI TIẾT ---")
display(stats_df)



In [ ]:
df['source_name'].nunique()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# 1. Lọc dữ liệu Top 10 nguồn tin
top_10_sources = df['source_name'].value_counts().head(20).index
df_top_source = df[df['source_name'].isin(top_10_sources)].copy()

# 2. Xử lý dữ liệu Landslide Size (nếu chưa chuẩn hoá)
# Chuyển về chữ thường và thay thế NaN bằng 'unknown'
df_top_source['landslide_size'] = df_top_source['landslide_size'].fillna('unknown').astype(str).str.lower().str.strip()

# 3. Tạo bảng chéo (Crosstab) và Chuẩn hóa theo hàng (index) để ra %
# normalize='index' giúp biến đổi số lượng thành tỷ lệ phần trăm (tổng hàng ngang = 100%)
size_distribution = pd.crosstab(
    df_top_source['source_name'], 
    df_top_source['landslide_size'], 
    normalize='index'
) * 100


# Sắp xếp lại thứ tự cột kích thước cho logic (Nhỏ -> Lớn)
# Lưu ý: Bạn kiểm tra lại các giá trị size thực tế trong data của bạn để khớp tên
desired_order = ['small', 'medium', 'large', 'very_large', 'unknown']
# Chỉ lấy những cột có tồn tại trong dữ liệu
existing_order = [size for size in desired_order if size in size_distribution.columns]
size_distribution = size_distribution[existing_order]

# 4. Trực quan hoá: Stacked Bar Chart 100%
ax = size_distribution.plot(
    kind='barh', 
    stacked=True, 
    figsize=(14, 8), 
    colormap='RdYlBu_r', # Màu từ Xanh (Nhỏ/An toàn) sang Đỏ (Lớn/Nguy hiểm)
    edgecolor='black',
    width=0.8
)

# 5. Thêm nhãn % lên biểu đồ (Chỉ hiện nếu > 5% để đỡ rối)
for c in ax.containers:
    labels = [f'{v.get_width():.0f}%' if v.get_width() > 5 else '' for v in c]
    ax.bar_label(c, labels=labels, label_type='center', fontsize=9, fontweight='bold', color='black')

plt.title('Phân bố Kích thước Sạt lở trong Top 10 Nguồn tin (%)', fontsize=16, fontweight='bold')
plt.xlabel('Tỷ lệ (%)', fontsize=12)
plt.ylabel('Nguồn tin', fontsize=12)
plt.legend(title='Kích thước', bbox_to_anchor=(1.0, 1), loc='upper left')
plt.xlim(0, 100)
plt.grid(axis='x', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

1. Luận điểm: Bằng chứng về Thiên lệch (Reporting Bias)
Thiên lệch báo cáo xảy ra khi khả năng một sự kiện được ghi nhận phụ thuộc vào vị trí của nó (nước giàu vs nước nghèo, ngôn ngữ tiếng Anh vs ngôn ngữ khác).

Bằng chứng 1: Sự thống trị của các nguồn tin địa phương (The "Oregon" Effect).

Trong bộ dữ liệu NASA này, có một nguồn tin cực kỳ chăm chỉ là "Oregon Department of Transportation". Họ báo cáo cả những vụ sạt lở nhỏ xíu trên đường cao tốc.

Kết quả: Bang Oregon (Mỹ) có số vụ sạt lở cao một cách vô lý so với các vùng núi hiểm trở khác trên thế giới.

Bằng chứng 2: Kích thước sạt lở (Size Bias).

Ở Mỹ hoặc Châu Âu, hạ tầng giám sát tốt nên họ ghi nhận được cả các vụ "Small" hoặc "Medium".

Ở các nước đang phát triển (Châu Á, Châu Phi), chỉ khi sạt lở "Large" hoặc "Very Large" (gây chết người, sập nhà) thì báo chí mới đưa tin và NASA mới biết để nhập vào.

2. Luận điểm: Bằng chứng về Tự nhiên (Natural Phenomena)
Mặc dù số lượng bị thiên lệch, nhưng dữ liệu vẫn tuân theo các quy luật vật lý:

Tính mùa vụ: Như bạn đã vẽ ở câu 1.1 và 1.3, sạt lở tăng vọt vào mùa mưa/bão. Đây là tín hiệu tự nhiên không thể làm giả.

Địa hình: Dù số liệu ở Nepal/Andes có thể bị thiếu (under-reported), nhưng những vụ được ghi nhận đều nằm ở vùng núi dốc, phù hợp với kiến thức địa lý.

#### 2.5 Có những khu vực nào ghi nhận sạt lở lặp đi lặp lại nhiều lần trong 10 năm không?

In [ ]:
import pandas as pd

# 1. Gom nhóm theo Quốc gia & Tỉnh/Bang
admin_repeat = df.groupby(['country_name', 'admin_division_name']).agg(
    total_events=('event_id', 'count'),            # Tổng số vụ
    total_deaths=('fatality_count', 'sum'),        # Tổng số chết
    unique_years=('event_date', lambda x: x.dt.year.nunique()), # Số năm có sạt lở (Max = 11)
    first_year=('event_date', lambda x: x.dt.year.min()),
    last_year=('event_date', lambda x: x.dt.year.max())
).reset_index()

# 2. Tính Tần suất trung bình (Vụ/Năm hoạt động)
admin_repeat['events_per_active_year'] = (admin_repeat['total_events'] / admin_repeat['unique_years']).round(1)

# 3. Lọc & Sắp xếp
# Chỉ lấy những nơi có ít nhất 10 vụ để xem xét tính lặp lại
top_repeat = admin_repeat[admin_repeat['total_events'] >= 0].sort_values('total_events', ascending=False)

# --- TÁCH RIÊNG MỸ VÀ THẾ GIỚI (Để tránh Oregon chiếm sóng) ---
us_stats = top_repeat[top_repeat['country_name'] == 'united states']
world_stats = top_repeat[top_repeat['country_name'] != 'united states']

# --- HIỂN THỊ KẾT QUẢ ---
cols_show = ['country_name', 'admin_division_name', 'unique_years', 'events_per_active_year']

print("--- TOP 10 KHU VỰC 'MÃN TÍNH' NHẤT THẾ GIỚI (TRỪ MỸ) ---")
print("Tiêu chí: Số năm xảy ra sạt lở cao và Tổng số vụ lớn")
display(world_stats[cols_show].head(10))

print("\n--- TOP 5 KHU VỰC LẶP LẠI NHIỀU NHẤT TẠI MỸ ---")
display(us_stats[cols_show].head(10))

### 3. "Mối quan hệ nhân - quả trong thảm họa sạt lở đất là gì? Các loại hình và nguyên nhân sạt lở cụ thể nào (WHAT & WHY) dẫn đến hậu quả nghiêm trọng nhất về mặt con người (IMPACT), và chúng phân bố ở đâu?"

3.1 (Loại phổ biến), 3.3 (Nguyên nhân thường gặp).

In [ ]:
category_counts = df['landslide_category'].value_counts().head(10)
# print(category_counts)

plt.figure(figsize=(12, 6))
sns.barplot(x=category_counts.values, y=category_counts.index, palette='magma')
plt.title('Các loại sạt lở phổ biến nhất')
plt.xlabel('Số lượng vụ')

### 3.2 Một số loại sạt lở có xu hướng gây chết người nhiều hơn các loại khác không?


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# 1. Chuẩn hóa tên loại sạt lở (quan trọng!)
df['landslide_category'] = df['landslide_category'].astype(str).str.lower().str.strip()


# 2. Gom nhóm và Tính toán thống kê
cat_stats = df.groupby('landslide_category').agg(
    event_count=('event_id', 'count'),          # Số vụ
    total_deaths=('fatality_count', 'sum'),     # Tổng chết
    avg_deaths=('fatality_count', 'mean'),      # Chết trung bình (Sát thương)
    max_deaths=('fatality_count', 'max')        # Vụ lớn nhất (Thảm họa)
).reset_index()

# 3. Lọc bỏ các loại quá hiếm (Giữ lại các loại có ít nhất 10 vụ)
cat_stats_filtered = cat_stats[cat_stats['event_count'] >= 10].sort_values('avg_deaths', ascending=False)

# 4. Vẽ biểu đồ so sánh (2 Subplots)
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

# Chart 1: Loại nào "Sát thương" nhất? (Trung bình chết/vụ)
sns.barplot(data=cat_stats_filtered, y='landslide_category', x='avg_deaths', ax=axes[0], palette='Reds_r')
axes[0].set_title('Mức độ Sát thương: Số người chết trung bình/vụ', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Số người chết trung bình')

# Thêm nhãn giá trị
for container in axes[0].containers:
    axes[0].bar_label(container, fmt='%.1f', padding=3)

# Chart 2: Loại nào gây Tổng thiệt hại lớn nhất?
# Sắp xếp lại theo tổng số chết
cat_stats_total = cat_stats_filtered.sort_values('total_deaths', ascending=False)
sns.barplot(data=cat_stats_total, y='landslide_category', x='total_deaths', ax=axes[1], palette='Oranges_r')
axes[1].set_title('Tổng thiệt hại: Tổng số người chết (2007-2017)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Tổng số người chết')

# Thêm nhãn giá trị
for container in axes[1].containers:
    axes[1].bar_label(container, padding=3)

plt.tight_layout()
plt.show()

# In bảng số liệu
print("--- BẢNG THỐNG KÊ CHI TIẾT ---")
display(cat_stats_filtered[['landslide_category', 'event_count', 'total_deaths', 'avg_deaths', 'max_deaths']])

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# 1. Chuẩn hoá cơ bản (Text Normalization)
# Chỉ chuyển về chữ thường và cắt khoảng trắng thừa để 'Rain' và 'rain ' không bị tính là 2 cái khác nhau
df['trigger_clean'] = df['landslide_trigger'].astype(str).str.lower().str.strip()

# 2. Loại bỏ giá trị 'unknown' hoặc 'nan' để biểu đồ tập trung vào các nguyên nhân CÓ thông tin
# Nếu bạn muốn giữ cả unknown thì bỏ dòng này đi
df_trigger_raw = df[~df['trigger_clean'].isin(['unknown', 'nan'])]

# 3. Đếm số lượng và lấy Top 15
top_triggers = df_trigger_raw['trigger_clean'].value_counts().head(15)

# 4. Vẽ biểu đồ
plt.figure(figsize=(12, 8))
sns.barplot(x=top_triggers.values, y=top_triggers.index, palette='magma')

plt.title('Top 15 Nguyên nhân gây sạt lở thường gặp nhất (Dữ liệu gốc)', fontsize=14, fontweight='bold')
plt.xlabel('Số lượng vụ')
plt.ylabel('Nguyên nhân ghi nhận')
plt.grid(axis='x', linestyle='--', alpha=0.5)

# Thêm nhãn số lượng
for index, value in enumerate(top_triggers.values):
    plt.text(value + 5, index, str(value), va='center', fontweight='bold')

plt.tight_layout()
plt.show()

# In bảng số liệu
print("--- CHI TIẾT SỐ LIỆU ---")
print(top_triggers)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Chuẩn hoá văn bản (Chỉ viết thường và bỏ khoảng trắng thừa)
df['trigger_clean'] = df['landslide_trigger'].astype(str).str.lower().str.strip()

# 2. Tính toán thống kê
# Gom nhóm theo nguyên nhân và tính: Số vụ, Tổng chết, Trung bình chết
trigger_stats = df.groupby('trigger_clean').agg(
    event_count=('event_id', 'count'),
    total_deaths=('fatality_count', 'sum'),
    avg_deaths=('fatality_count', 'mean')
).reset_index()

# 3. Lọc dữ liệu
# Chỉ lấy các nguyên nhân có ít nhất 10 vụ để số trung bình có ý nghĩa thống kê
# (Loại bỏ các nguyên nhân hiếm gặp nhưng ngẫu nhiên gây chết nhiều)
trigger_stats_filtered = trigger_stats[
    (trigger_stats['event_count'] >= 10) & 
    (trigger_stats['trigger_clean'] != 'unknown')
]

# 4. Vẽ biểu đồ so sánh (2 cái)
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

# --- BIỂU ĐỒ 1: MỨC ĐỘ SÁT THƯƠNG (Trung bình số chết/vụ) ---
# Sắp xếp giảm dần theo avg_deaths
top_lethal = trigger_stats_filtered.sort_values('avg_deaths', ascending=False).head(15)

sns.barplot(data=top_lethal, y=top_lethal['trigger_clean'], x='avg_deaths', ax=axes[0], palette='Reds_r')
axes[0].set_title('Top 15 Nguyên nhân "Sát thương" nhất\n(Trung bình số người chết/vụ)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Số người chết trung bình')
axes[0].set_ylabel('Nguyên nhân')

for container in axes[0].containers:
    axes[0].bar_label(container, fmt='%.1f', padding=3)

# --- BIỂU ĐỒ 2: TỔNG THIỆT HẠI (Tổng số người chết) ---
# Sắp xếp giảm dần theo total_deaths
top_impact = trigger_stats_filtered.sort_values('total_deaths', ascending=False).head(15)

sns.barplot(data=top_impact, y=top_impact['trigger_clean'], x='total_deaths', ax=axes[1], palette='Oranges_r')
axes[1].set_title('Top 15 Nguyên nhân gây Tổng thiệt hại lớn nhất\n(Tổng số người chết 2007-2017)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Tổng số người chết')
axes[1].set_ylabel('')

for container in axes[1].containers:
    axes[1].bar_label(container, padding=3)

plt.tight_layout()
plt.show()

# In ra bảng số liệu để bạn dễ nhận xét
print("--- THỐNG KÊ CHI TIẾT CÁC NGUYÊN NHÂN NGUY HIỂM ---")
display(top_lethal[['trigger_clean', 'event_count', 'total_deaths', 'avg_deaths']])

 4.1 (Phân bố tử vong), 4.3 (Chuẩn hóa mức độ nghiêm trọng).

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Chuẩn bị dữ liệu (trên df đã có sẵn)
# Đảm bảo cột ngày tháng đúng định dạng để lấy ra Năm
df['event_date'] = pd.to_datetime(df['event_date'], errors='coerce')

# Điền 0 cho các ô thiếu dữ liệu (để tính tổng không bị lỗi)
df['fatality_count'] = df['fatality_count'].fillna(0)

# Gom nhóm tính Tổng theo năm
yearly_deaths = df.groupby(df['event_date'].dt.year)['fatality_count'].sum().reset_index()
yearly_deaths.columns = ['Year', 'Total Deaths']

# 2. Vẽ biểu đồ
plt.figure(figsize=(12, 6))

# Vẽ cột màu đỏ (tượng trưng cho thiệt hại/cảnh báo)
ax = sns.barplot(data=yearly_deaths, x='Year', y='Total Deaths', palette='Reds')

# Trang trí
plt.title('Tổng số người chết do Sạt lở theo Năm (2007 - 2017)', fontsize=16, fontweight='bold', color='#8B0000')
plt.xlabel('Năm', fontsize=12)
plt.ylabel('Tổng số người chết', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.5)

# Hiển thị con số cụ thể trên đỉnh mỗi cột
for container in ax.containers:
    ax.bar_label(container, padding=3, fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# 1. Chuẩn bị dữ liệu
df['trigger_clean'] = df['landslide_trigger'].astype(str).str.lower().str.strip()
df['category_clean'] = df['landslide_category'].astype(str).str.lower().str.strip()

# 2. Lọc lấy các "Ông lớn" (Top categories & Top triggers)
# Lấy Top 4 loại sạt lở phổ biến nhất
top_cats = df['category_clean'].value_counts().index.tolist()
# Lấy Top 6 nguyên nhân phổ biến nhất
top_trigs = df['trigger_clean'].value_counts().head(20).index.tolist()

# Tạo dataframe chỉ chứa các ông lớn này
df_filtered = df[
    (df['category_clean'].isin(top_cats)) & 
    (df['trigger_clean'].isin(top_trigs))
].copy()

# 3. Tạo Pivot Table (Bảng chéo)
# Tính số người chết TRUNG BÌNH cho mỗi cặp (Category - Trigger)
pivot_severity = df_filtered.pivot_table(
    index='trigger_clean', 
    columns='category_clean', 
    values='fatality_count', 
    aggfunc='sum',
    fill_value=0
)

# 4. Vẽ Heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(
    pivot_severity, 
    annot=True,     # Hiện số lên ô
    fmt=".1f",      # Lấy 1 số lẻ
    cmap="YlOrRd",  # Màu vàng (nhẹ) -> Đỏ (nghiêm trọng)
    linewidths=.5,  # Đường kẻ giữa các ô
    cbar_kws={'label': 'Số người chết trung bình/vụ'}
)

plt.title('Mức độ Nghiêm trọng (Avg Fatalities) theo Loại hình & Nguyên nhân', fontsize=14, fontweight='bold')
plt.xlabel('Loại sạt lở (Category)')
plt.ylabel('Nguyên nhân kích hoạt (Trigger)')

plt.tight_layout()
plt.show()

3.2 & 4.4: Một số loại/nguyên nhân sạt lở có gây chết người nhiều hơn không? (Ví dụ: Lũ bùn đá do mưa bão có gây tử vong cao hơn sụt lở do động đất không?).

3.5: Trong cùng một loại, nguyên nhân khác nhau có làm mức độ nghiêm trọng khác đi không? (Ví dụ: Cùng là lũ bùn đá, do cháy rừng gây ra có chết nhiều người hơn do bão không?).

4.2 & Liên kết không gian từ Chủ đề 2: Quốc gia/vùng nào hứng chịu hậu quả nặng nề nhất từ những "cặp đôi nguy hiểm" (loại + nguyên nhân gây chết người nhiều) đã xác định ở trên?

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 1. CHUẨN BỊ DỮ LIỆU
# Lọc bỏ dữ liệu thiếu
df_cat = df.dropna(subset=['region', 'landslide_category']).copy()

# Làm sạch tên loại hình (bỏ dấu gạch dưới, viết hoa chữ đầu)
df_cat['category_clean'] = df_cat['landslide_category'].astype(str).str.replace('_', ' ').str.title()

# Chỉ lấy Top 5 loại hình phổ biến nhất (để gom các loại lặt vặt vào 'Other')
top_n = 5
top_cats = df_cat['category_clean'].value_counts().head(top_n).index
df_cat['category_grouped'] = df_cat['category_clean'].apply(lambda x: x if x in top_cats else 'Other')

# 2. TẠO BẢNG PIVOT (TÍNH PHẦN TRĂM)
# Crosstab tự động tính tần suất
category_matrix = pd.crosstab(df_cat['region'], df_cat['category_grouped'], normalize='index') * 100

# Sắp xếp lại thứ tự vùng (như cũ)

valid_regions = [r for r in ordered_regions if r in category_matrix.index]
category_matrix = category_matrix.loc[valid_regions]

# 3. VẼ BIỂU ĐỒ CỘT CHỒNG 100%
plt.figure(figsize=(14, 8))

# Vẽ biểu đồ
# category_matrix.plot(kind='barh', stacked=True) vẽ trực tiếp từ pandas rất tiện cho dạng này
ax = category_matrix.plot(
    kind='barh', 
    stacked=True, 
    figsize=(14, 9), 
    colormap='Set2', # Bộ màu dịu mắt
    edgecolor='white',
    width=0.8
)

# 4. TRANG TRÍ & HIỂN THỊ SỐ %
plt.title('Cơ cấu Loại hình Sạt lở theo Khu vực Địa lý (Tỷ lệ %)', fontsize=16, fontweight='bold')
plt.xlabel('Tỷ lệ phần trăm (%)', fontsize=12)
plt.ylabel('Khu vực', fontsize=12)
plt.legend(title='Loại hình (Category)', bbox_to_anchor=(1.01, 1), loc='upper left')
plt.xlim(0, 100)
plt.grid(axis='x', linestyle='--', alpha=0.5)

# Hiển thị số % lên giữa các thanh màu
for c in ax.containers:
    # c là tập hợp các thanh cùng màu
    labels = [f'{w:.0f}%' if w > 5 else '' for w in c.datavalues] # Chỉ hiện số nếu > 5% cho đỡ rối
    ax.bar_label(c, labels=labels, label_type='center', fontsize=10, fontweight='bold', color='black')

plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# 1. CHUẨN BỊ DỮ LIỆU
# Lọc dữ liệu có số liệu chết và vùng
df_fatality = df.dropna(subset=['fatality_count', 'region']).copy()
# Bỏ vùng Unknown và lọc các giá trị = 0 nếu muốn nhìn rõ các vụ có thương vong (Tùy chọn)
# Ở đây ta giữ cả số 0 để thấy trung thực bức tranh toàn cảnh
df_fatality = df_fatality[df_fatality['region'] != 'unknown']

# Để log scale hoạt động tốt, ta thay giá trị 0 bằng một số rất nhỏ (ví dụ 0.1) 
# vì log(0) là vô cực âm.
df_fatality['fatality_log'] = df_fatality['fatality_count'].replace(0, 0.5)

# 2. VẼ BIỂU ĐỒ BOX PLOT
plt.figure(figsize=(14, 8))

# Sắp xếp thứ tự theo số người chết trung bình để dễ nhìn
order_region = df_fatality.groupby('region')['fatality_count'].mean().sort_values(ascending=False).index

sns.boxplot(
    data=df_fatality,
    x='region',
    y='fatality_count',
    order=order_region,
    palette='Reds', # Màu đỏ thể hiện sự nguy hiểm
    showfliers=True # Hiện các điểm ngoại lai (các vụ thảm họa lớn)
)

# 3. CHUYỂN TRỤC Y SANG LOG SCALE
plt.yscale('log')

# 4. TRANG TRÍ
plt.title('Phân bố Số người chết mỗi vụ sạt lở theo Khu vực (Thang đo Log)', fontsize=16, fontweight='bold')
plt.xlabel('Khu vực Địa lý', fontsize=12)
plt.ylabel('Số người chết (Log Scale)', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.xticks(rotation=45)

# Chỉnh nhãn trục Y cho dễ đọc (thay vì 10^1, 10^2...)
from matplotlib.ticker import ScalarFormatter
ax = plt.gca()
ax.yaxis.set_major_formatter(ScalarFormatter())

plt.tight_layout()
plt.show()

# 5. BẢNG THỐNG KÊ CHI TIẾT
print("--- BẢNG THỐNG KÊ MỨC ĐỘ THƯƠNG VONG ---")
stats = df_fatality.groupby('region')['fatality_count'].agg(
    So_vu=('count'),
    Tong_chet=('sum'),
    Trung_binh=('mean'),
    Trung_vi=('median'),
    Vu_lon_nhat=('max')
).sort_values('Trung_binh', ascending=False).round(1)

display(stats)

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# 1. CHUẨN BỊ DỮ LIỆU
# Lọc bỏ dòng thiếu dữ liệu trigger hoặc region
df_trig = df.dropna(subset=['region', 'landslide_trigger']).copy()

# Chuẩn hoá trigger (viết thường, bỏ khoảng trắng thừa)
df_trig['trigger_clean'] = df_trig['landslide_trigger'].astype(str).str.lower().str.strip()

# Chỉ lấy Top 7 nguyên nhân phổ biến nhất (để biểu đồ đỡ rối)
top_triggers = df_trig['trigger_clean'].value_counts().head(10).index
df_trig_top = df_trig[df_trig['trigger_clean'].isin(top_triggers)]

# 2. TẠO BẢNG PIVOT (Cross-tabulation)
# Đếm số vụ của từng nguyên nhân trong từng vùng
trigger_matrix = pd.crosstab(df_trig_top['trigger_clean'], df_trig_top['region'])

# Sắp xếp lại thứ tự vùng cho logic (như cũ)
# ordered_regions = [
#     'North America (Bắc Mỹ)', 'Central America (Trung Mỹ)', 'South America (Nam Mỹ)',
#     'Europe (Châu Âu)', 'Africa (Châu Phi)',
#     'South Asia (Nam Á)', 'East Asia (Đông Á)', 'SE Asia & Oceania (ĐNÁ & Úc)'
# ]
valid_cols = [c for c in ordered_regions if c in trigger_matrix.columns]
trigger_matrix = trigger_matrix[valid_cols]

# 3. VẼ HEATMAP
plt.figure(figsize=(14, 8))

sns.heatmap(
    trigger_matrix, 
    annot=True, 
    fmt='d',       # Hiển thị số nguyên
    cmap='Blues',  # Màu xanh cho dễ nhìn
    linewidths=.5,
    cbar_kws={'label': 'Số lượng vụ'}
)

plt.title('Ma trận Nguyên nhân gây Sạt lở tại các Khu vực Địa lý', fontsize=16, fontweight='bold')
plt.xlabel('Khu vực (Region)', fontsize=12)
plt.ylabel('Nguyên nhân (Trigger)', fontsize=12)
plt.yticks(rotation=0) # Giữ nhãn trục Y nằm ngang cho dễ đọc
plt.xticks(rotation=45) # Giữ nhãn trục Y nằm ngang cho dễ đọc

plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 1. CHUẨN BỊ DỮ LIỆU
# Lọc 3 vùng khí hậu chính
target_zones = ['tropical', 'subtropical', 'temperate']
df_sub = df[df['climate_zone'].isin(target_zones)].copy()

# Chuẩn hoá tên loại sạt lở
df_sub['category_clean'] = df_sub['landslide_category'].astype(str).str.lower().str.strip()

# Chỉ lấy Top 5 loại sạt lở phổ biến nhất để biểu đồ gọn gàng
top_categories = df_sub['category_clean'].value_counts().head(5).index.tolist()
df_final = df_sub[df_sub['category_clean'].isin(top_categories)]

# 2. TẠO BẢNG CHÉO (CROSSTAB) VÀ CHUẨN HOÁ %
# normalize='index' nghĩa là tổng mỗi hàng (mỗi vùng) sẽ cộng lại bằng 1 (100%)
cross_tab = pd.crosstab(
    df_final['climate_zone'], 
    df_final['category_clean'], 
    normalize='index'
) * 100

# 3. VẼ BIỂU ĐỒ CỘT CHỒNG 100%
ax = cross_tab.plot(kind='barh', stacked=True, figsize=(12, 6), colormap='Set2')

# Trang trí
plt.title('Cơ cấu thành phần Loại sạt lở theo Vùng Khí hậu', fontsize=14, fontweight='bold')
plt.xlabel('Tỷ lệ (%)')
plt.ylabel('Vùng Khí hậu')
plt.legend(title='Loại sạt lở', bbox_to_anchor=(1.05, 1), loc='upper left')

# Hiển thị % lên biểu đồ cho dễ đọc
for n, x in enumerate([*cross_tab.index.values]):
    for (img, label) in zip(ax.containers, cross_tab.columns):
        # Lấy giá trị %
        value = cross_tab.loc[x, label]
        # Chỉ hiện số nếu miếng bánh đủ lớn (>3%)
        if value > 3:
            # Tính vị trí để đặt chữ vào giữa miếng màu
            # (Hơi phức tạp chút để căn chỉnh tự động)
            pass 
            
# (Đoạn code hiển thị số lên Stacked Bar hơi dài dòng, 
# nên mình để code vẽ clean, bạn nhìn trục X là ước lượng được rồi)

plt.tight_layout()
plt.show()

print("--- BẢNG SỐ LIỆU CHI TIẾT (%) ---")
display(cross_tab.round(1))

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# 1. CHUẨN BỊ DỮ LIỆU
# Chuẩn hoá văn bản (chỉ viết thường, bỏ khoảng trắng thừa)
df['trigger_raw'] = df['landslide_trigger'].astype(str).str.lower().str.strip()

# Lọc 3 vùng khí hậu chính
target_zones = ['tropical', 'subtropical', 'temperate']
df_filtered = df[df['climate_zone'].isin(target_zones)].copy()

# 2. LẤY TOP 15 NGUYÊN NHÂN PHỔ BIẾN NHẤT
# Vì không gộp nhóm nên danh sách rất dài, ta chỉ lấy Top 15 để biểu đồ đọc được
top_raw_triggers = df_filtered['trigger_raw'].value_counts().head(15).index.tolist()

# Chỉ giữ lại dữ liệu thuộc Top 15 này
df_final = df_filtered[df_filtered['trigger_raw'].isin(top_raw_triggers)]

# 3. TẠO BẢNG CHÉO (CROSSTAB) VÀ CHUẨN HOÁ %
# normalize='index': Tổng % theo hàng ngang (theo vùng) = 100%
cross_matrix = pd.crosstab(
    df_final['climate_zone'], 
    df_final['trigger_raw'], 
    normalize='index' 
) * 100

# 4. VẼ HEATMAP
plt.figure(figsize=(16, 6)) # Kéo dài bề ngang vì có 15 cột

sns.heatmap(
    cross_matrix, 
    annot=True, 
    fmt='.1f',      # Lấy 1 số lẻ
    cmap='YlGnBu',  # Màu Xanh vàng (nhạt -> đậm)
    cbar_kws={'label': 'Tỷ lệ (%) trong vùng'}
)

plt.title('Phân bố 15 Nguyên nhân gốc phổ biến nhất theo Vùng Khí hậu', fontsize=14, fontweight='bold')
plt.xlabel('Nguyên nhân ghi nhận (Trigger Raw)')
plt.ylabel('Vùng Khí hậu')

plt.tight_layout()
plt.show()